In [12]:
from ising_benchmark.utils import ising_generator

N = 150

# Sample from normal distribution
h, J = ising_generator(N)

Assuming the following optimization problem

$$

\min \sum_i h_i \sigma_i + \sum_i \sum_{i \ne j} J_{ij}\sigma_i \sigma_j

$$

we would like to find its minimum eigenvalue and associated eignevector.

In [13]:
import neal
import dimod
from ising_benchmark.utils import ising_to_bqm, bqm_spin_array

bqm = ising_to_bqm(h, J)

if N < 20:
    sampleset = dimod.ExactSolver().sample(bqm)

    exact_e = bqm.energy(sampleset.first.sample)
    exact_solution = bqm_spin_array(sampleset.first.sample)

    print(f"Obtained energy: {exact_e}")

In [14]:
import os
from dotenv import load_dotenv
from ising_benchmark.utils import submit_dwave

load_dotenv()

sampleset = submit_dwave(bqm, os.environ["TOKEN"])

dwave_e = bqm.energy(sampleset.first.sample)
dwave_solution = bqm_spin_array(sampleset.first.sample)

print(f"Obtained energy (DWave): {dwave_e}")

Obtained energy (DWave): -2669.0511325516413


In [15]:
sampler = neal.SimulatedAnnealingSampler()
# Run with default parameters
sampleset = sampler.sample(bqm)

sa_e = bqm.energy(sampleset.first.sample)
sa_solution = bqm_spin_array(sampleset.first.sample)

print(f"Obtained energy (SA): {sa_e}")

Obtained energy (SA): -2652.628423454082


In [16]:
# Run with specified parameters
sampleset = sampler.sample(bqm, seed=1234, beta_range=[0.1, 4.2], num_sweeps=20, beta_schedule_type='geometric')

geom_e = bqm.energy(sampleset.first.sample)
geom_solution = bqm_spin_array(sampleset.first.sample)

print(f"Obtained energy (SA): {geom_e}")

Obtained energy (SA): -2377.3026423359083


In [17]:
import torch
import simulated_bifurcation as sb

h_torch = torch.tensor(h, dtype=torch.float32)
J_torch = torch.tensor(J, dtype=torch.float32)

# Binary minimization
sb_solution, sb_e = sb.minimize(J_torch, h_torch, input_type='spin')

print(f"Obtained energy (SB): {sb_e}")

Bifurcated agents: 100%|██████████| 128/128 [00:10<00:00, 12.71it/s]

Obtained energy (SB): -2669.0517578125


In [18]:
dwave_solution == sb_solution

False